In [1]:
import zipfile
import os

with zipfile.ZipFile(os.path.join(os.getcwd(),'cifar10.zip'),'r') as zip_ref:
    zip_ref.extractall()

In [10]:
import torch
from torch import nn

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from PIL import  Image
import pathlib

In [3]:
train_dir = os.path.join(os.getcwd(),'cifar10','train')
test_dir  = os.path.join(os.getcwd(),'cifar10','test')


In [5]:
class_names = [name for name in os.listdir(test_dir)]
class_names.sort()
print(class_names)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [6]:
class_dict = {num:name for num,name in enumerate(class_names)}
print(class_dict)

{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


<bound method Image.show of <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x71D484F42590>>

In [14]:
class cifarDS(Dataset):

    def __init__(self,path:str,transform:transforms = None):
        self.path = path
        self.transform = transform
        self.all_images = pathlib.Path(self.path).glob('*/*.png')


    def __len__(self):
        return len(self.all_images)
    
    def getImage(self,path:str) -> Image.Image: 

        return Image.open(path)

    def __getitem__(self, index:str ) -> torch.Tensor:

        path = self.all_images[index]

        image = self.getImage(path)

        if self.transform:
            return self.transform(image)
        

        else:
            self.transform = transforms.Compose([
                transforms.ToTensor()
            ])







        